In [1]:
import sys
sys.path.append(r'C:\Users\kuina\OneDrive\TFG\Codigo')

In [2]:
from datasets import load_dataset
import json

In [3]:
map_about = {0:'ABOUT:female',1:'ABOUT:male'}
rev_about = {v:k for k,v in map_about.items()}
map_self = {0:'SELF:female',1:'SELF:male'}
map_partner = {0:'PARTNER:female',1:'PARTNER:male'}

In [4]:
list(map(map_about.get,[0,0,1]))

['ABOUT:female', 'ABOUT:female', 'ABOUT:male']

## Datos Huggingface

In [5]:
convai2_about = load_dataset('md_gender_bias','convai2_inferred')
train_about = convai2_about['train']
print(train_about[0])

Reusing dataset md_gender_bias (C:\Users\kuina\.cache\huggingface\datasets\md_gender_bias\convai2_inferred\1.0.0\8ae77b51acf93383161cc954b146159291beca6c979b54ce228c46db86116c05)


  0%|          | 0/3 [00:00<?, ?it/s]

{'text': "hi , how are you doing ? i'm getting ready to do some cheetah chasing to stay in shape .", 'binary_label': 1, 'binary_score': 0.6521999835968018, 'ternary_label': 2, 'ternary_score': 0.4496000111103058}


In [6]:
train_about = train_about.remove_columns(['binary_score', 'ternary_label', 'ternary_score'])

In [7]:
train_about

Dataset({
    features: ['text', 'binary_label'],
    num_rows: 131438
})

In [8]:
#Filtro repetidos para que sean una única entrada
data = {}

for item in train_about:
    text = item['text']
    label = item['binary_label']
    if text not in data:
        # data[text] =[label]
        data[text] =[map_about[label]]
    else:
        data[text].append(map_about[label])

In [9]:
#Calculo probabilidades

for item in data:
    labels = data[item]

    prob_labels = {label : labels.count(label)/len(labels) for label in set(labels)}
    data[item] = prob_labels

In [10]:
len(data)

115877

In [11]:
data

{"hi , how are you doing ? i'm getting ready to do some cheetah chasing to stay in shape .": {'ABOUT:male': 1.0},
 'i am ! for my hobby i like to do canning or some whittling .': {'ABOUT:male': 1.0},
 "that's neat . when i was in high school i placed 6th in 100m dash !": {'ABOUT:male': 1.0},
 'i do not . but i do have a favorite meat since that is all i eat exclusively .': {'ABOUT:female': 1.0},
 'i would have to say its prime rib . do you have any favorite foods ?': {'ABOUT:female': 1.0},
 'do you have anything planned for today ? i think i am going to do some canning .': {'ABOUT:female': 1.0},
 'i think i will can some jam . do you also play footfall for fun ?': {'ABOUT:male': 1.0},
 'hi , how are you doing today ?': {'ABOUT:male': 1.0},
 'wow , four sisters . just watching game of thrones .': {'ABOUT:female': 1.0},
 'i agree . what do you do for a living ?': {'ABOUT:male': 1.0},
 "interesting . i'm a website designer . pretty much spend all my time on the computer .": {'ABOUT:female

## Carga datos ParlAI

In [12]:
with open(r'./convai_two_NO_unk.json') as f:
    convai2_parlai = json.load(f)

In [13]:
convai2_parlai

{"the times . i'm a carnivore , how about you ?": {'partner': {'PARTNER:male': 1.0},
  'self': {'SELF:male': 1.0}},
 'sounds fancy . any cool perks ?': {'self': {'SELF:female': 1.0},
  'partner': {'PARTNER:female': 0.5, 'PARTNER:male': 0.5}},
 'that is not embarrassing in this economy': {'self': {'SELF:female': 0.5,
   'SELF:male': 0.5},
  'partner': {'PARTNER:female': 0.5, 'PARTNER:male': 0.5}},
 'no . what is that ? ?': {'self': {'SELF:female': 1.0},
  'partner': {'PARTNER:female': 1.0}},
 'person i am sorry . do you do anything active or are you too tired fo that ?': {'self': {'SELF:female': 0.5,
   'SELF:male': 0.5},
  'partner': {'PARTNER:female': 0.5, 'PARTNER:male': 0.5}},
 'i see i enjoy reading american literature what do you enjoy ?': {'partner': {'PARTNER:female': 0.5,
   'PARTNER:male': 0.5},
  'self': {'SELF:female': 0.5, 'SELF:male': 0.5}},
 'cool . cities are great , i prefer the beach .': {'partner': {'PARTNER:female': 0.5,
   'PARTNER:male': 0.5},
  'self': {'SELF:fema

# Unión de los datasets

Itero sobre convai porque tiene menos elementos (no deberia hacerme falta comprobar que ya los haya incluido porque es algo que tuve en cuenta con los datasets individuales, ahroa solo tengo qeu iterar sobre uno y ver si la frase tiene etiqueta en el otro)

In [14]:
join_data = {}
for item in convai2_parlai:
    if item in data:
        join_data[item] = {'label_about':data[item],
                            'label_to':convai2_parlai[item]['partner'],
                            'label_as':convai2_parlai[item]['self']}


In [15]:
join_data

{"the times . i'm a carnivore , how about you ?": {'label_about': {'ABOUT:male': 1.0},
  'label_to': {'PARTNER:male': 1.0},
  'label_as': {'SELF:male': 1.0}},
 'sounds fancy . any cool perks ?': {'label_about': {'ABOUT:male': 1.0},
  'label_to': {'PARTNER:female': 0.5, 'PARTNER:male': 0.5},
  'label_as': {'SELF:female': 1.0}},
 'that is not embarrassing in this economy': {'label_about': {'ABOUT:female': 1.0},
  'label_to': {'PARTNER:female': 0.5, 'PARTNER:male': 0.5},
  'label_as': {'SELF:female': 0.5, 'SELF:male': 0.5}},
 'no . what is that ? ?': {'label_about': {'ABOUT:male': 1.0},
  'label_to': {'PARTNER:female': 1.0},
  'label_as': {'SELF:female': 1.0}},
 'person i am sorry . do you do anything active or are you too tired fo that ?': {'label_about': {'ABOUT:male': 1.0},
  'label_to': {'PARTNER:female': 0.5, 'PARTNER:male': 0.5},
  'label_as': {'SELF:female': 0.5, 'SELF:male': 0.5}},
 'i see i enjoy reading american literature what do you enjoy ?': {'label_about': {'ABOUT:male': 1.0

In [16]:
with open('convai_all_labels_probs.json','w') as f:
    json.dump(join_data,f)

# Alternativa: parsing solo con Stanza

In [17]:
from dependency_utils import create_dependency_tags

In [18]:
with open('convai_all_labels_probs.json','r') as f:
    data = json.load(f)

In [19]:
create_dependency_tags(data,'relative','_')

Comenzando parsing de dependencias....

Parsing de dependencias terminado. Duración del proceso: 3.1812148412068684 minutos
Comenzando a generar el encoding para las dependencias


100%|██████████| 73823/73823 [03:42<00:00, 331.06it/s]

Proceso terminado con éxito


In [20]:
with open('convai_rel_complete.json','w') as f:
    json.dump(data,f)